In [39]:
import torch
import onnx
from brevitas.export import export_qonnx

from finn.util.visualization import showInNetron

from finn.util.test import get_test_model_trained
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_datatypes import InferDataTypes

import numpy as np
import qonnx.core.onnx_exec as oxe

import cv2

### Export to QONNX during training and test with DFire MINI

In [19]:
ori_onnx = 'BED_classifier__best_mean_F1__perChannel_fxPoint__QONNX.onnx'
clean_onnx = 'BED_classifier__best_mean_F1_clean__perChannel_fxPoint__QONNX.onnx'

In [20]:
qonnx_cleanup(ori_onnx, out_file=clean_onnx)

In [21]:
showInNetron(ori_onnx)

Stopping http://0.0.0.0:8083
Serving 'BED_classifier__best_mean_F1__perChannel_fxPoint__QONNX.onnx' at http://0.0.0.0:8083


In [22]:
showInNetron(clean_onnx)

Stopping http://0.0.0.0:8083
Serving 'BED_classifier__best_mean_F1_clean__perChannel_fxPoint__QONNX.onnx' at http://0.0.0.0:8083


In [23]:
finn_tidy_onnx =  'BED_classifier__best_mean_F1_finn_tidy__perChannel_fxPoint__QONNX.onnx'

In [24]:
model = ModelWrapper(clean_onnx)
model = model.transform(ConvertQONNXtoFINN())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(finn_tidy_onnx)

In [25]:
showInNetron(finn_tidy_onnx)

Stopping http://0.0.0.0:8083
Serving 'BED_classifier__best_mean_F1_finn_tidy__perChannel_fxPoint__QONNX.onnx' at http://0.0.0.0:8083


# Compare Outputs

In [26]:
dummy_in = np.random.randint(low = 0, high = 255+1, size = (1, 3, 224, 224)) 
dummy_in = (dummy_in / 256.).astype(np.float32)

In [27]:
#ori_model = ModelWrapper(ori_onnx)
clean_model = ModelWrapper(clean_onnx)
finn_model = ModelWrapper(finn_tidy_onnx)

### Cleaned QONNX

In [28]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(clean_model, input_dict)
produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
produced_clean_qonnx

array([[-1.3982027, -2.1908169]], dtype=float32)

### FINN QONNX

In [29]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(finn_model, input_dict)
produced_finn_qonnx = output_dict[list(output_dict.keys())[0]]
produced_finn_qonnx

array([[-1.3982027, -2.1908169]], dtype=float32)

# Compare with Images

In [30]:
img_file = './WEB10495.jpg' # Smoke and Fire
#img_file = './WEB10980.jpg' # Only Smoke
#img_file = './WEB10031.jpg' # Empty
img = cv2.imread(img_file)
img = cv2.imread(img_file)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
img = cv2.resize(img, (224,224), interpolation = cv2.INTER_LINEAR) 
img = img / 256.
img = np.expand_dims(img, axis=0)
img = np.transpose(img, (0, 3, 1, 2))
img = img.astype(np.float32)

In [31]:
print(f'Image shape: {img.shape}')
print(f'Data type: {img.dtype}')

Image shape: (1, 3, 224, 224)
Data type: float32


### Cleaned QONNX

In [32]:
input_dict = {"global_in": img}
output_dict = oxe.execute_onnx(clean_model, input_dict)
produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
produced_clean_qonnx

array([[0.6415325 , 0.50019217]], dtype=float32)

### FINN QONNX

In [33]:
input_dict = {"global_in": img}
output_dict = oxe.execute_onnx(finn_model, input_dict)
produced_finn_qonnx = output_dict[list(output_dict.keys())[0]]
produced_finn_qonnx

array([[0.6415325 , 0.50019217]], dtype=float32)

# Add Preprocessing Node: /255. -> ToTensor, FINN function

In [34]:
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType

In [40]:
model = ModelWrapper(finn_tidy_onnx)
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = "./preproc_BED_input.onnx"
export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name)
qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = pre_model.transform(ConvertQONNXtoFINN())

# join preprocessing and core model
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/merge_onnx_models.py:70: UserWarning: [MergeONNXModels] opsets for models to merge differ: 14 vs 17, output model will use opset 17
  warnings.warn(
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


### Tidy up again and save

In [41]:
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

finn_tidy_preprocess = 'BED_classifier__best_mean_F1_finn_prepro__perChannel_fxPoint__QONNX.onnx'
model.save(finn_tidy_preprocess)